In [2]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
ACCOUNT_ID = os.getenv("ACCOUNT_ID")
URL = os.getenv("OANDA_URL")
SECURE_HEADER = {
    'Authorization': f'Bearer {API_KEY}'
}

#### Fetch data from oanda

In [4]:
def fetch_data(instrument="AUD_CHF", granularity="M15", count=5000):
    session = requests.Session()

    params = {
        "count": count,
        "granularity": granularity,
    }

    url = f"{URL}/instruments/{instrument}/candles"
    response = session.get(url, params=params, headers=SECURE_HEADER) 
    
    return response.json()

In [20]:
# Test fetch data function
raw_data = fetch_data("EUR_USD","M1", 5000)
raw_data

{'instrument': 'EUR_USD',
 'granularity': 'M1',
 'candles': [{'complete': True,
   'volume': 87,
   'time': '2025-01-03T20:20:00.000000000Z',
   'mid': {'o': '1.03060', 'h': '1.03064', 'l': '1.03048', 'c': '1.03053'}},
  {'complete': True,
   'volume': 59,
   'time': '2025-01-03T20:21:00.000000000Z',
   'mid': {'o': '1.03051', 'h': '1.03054', 'l': '1.03048', 'c': '1.03053'}},
  {'complete': True,
   'volume': 50,
   'time': '2025-01-03T20:22:00.000000000Z',
   'mid': {'o': '1.03055', 'h': '1.03062', 'l': '1.03052', 'c': '1.03062'}},
  {'complete': True,
   'volume': 29,
   'time': '2025-01-03T20:23:00.000000000Z',
   'mid': {'o': '1.03061', 'h': '1.03062', 'l': '1.03056', 'c': '1.03057'}},
  {'complete': True,
   'volume': 88,
   'time': '2025-01-03T20:24:00.000000000Z',
   'mid': {'o': '1.03056', 'h': '1.03058', 'l': '1.03046', 'c': '1.03047'}},
  {'complete': True,
   'volume': 63,
   'time': '2025-01-03T20:25:00.000000000Z',
   'mid': {'o': '1.03046', 'h': '1.03052', 'l': '1.03046',

#### Converting data into dataframe

In [21]:
def convert_to_df(data):
    hist_data = []
    for candle in data['candles']:
        if candle['complete']:
            hist = dict(
                time=candle['time'],
                volume=candle['volume'],
                open=float(candle['mid']['o']),
                low=float(candle['mid']['l']),
                high=float(candle['mid']['h']),
                close=float(candle['mid']['c']),
            )
            hist_data.append(hist)

    df = pd.DataFrame(hist_data)
    return df

In [24]:
# Test convert to df function
data = convert_to_df(raw_data)
data

,time,volume,open,low,high,close
0,2025-01-03T20:20:00.000000000Z,87,1.03060,1.03048,1.03064,1.03053
1,2025-01-03T20:21:00.000000000Z,59,1.03051,1.03048,1.03054,1.03053
2,2025-01-03T20:22:00.000000000Z,50,1.03055,1.03052,1.03062,1.03062
3,2025-01-03T20:23:00.000000000Z,29,1.03061,1.03056,1.03062,1.03057
4,2025-01-03T20:24:00.000000000Z,88,1.03056,1.03046,1.03058,1.03047
...,...,...,...,...,...,...
4994,2025-01-09T08:06:00.000000000Z,246,1.02884,1.02878,1.02920,1.02904
4995,2025-01-09T08:07:00.000000000Z,201,1.02902,1.02896,1.02924,1.02912
4996,2025-01-09T08:08:00.000000000Z,163,1.02912,1.02897,1.02918,1.02898
4997,2025-01-09T08:09:00.000000000Z,218,1.02897,1.02897,1.02930,1.02924


In [7]:
def is_trade(row):
    try:
        if row['cross'] == 1 and row['close'] > row['EMA_200']: # close > ema200 is uptrend
            isTrade = True
            stoploss = row['last_low']
            lossdept = abs(row['close'] - stoploss)
            takeprofit = max(row['last_high'], row['close'] + 2 * lossdept)  # Ensure TP is 2x SL
            
            return isTrade, takeprofit, stoploss

        if row['cross'] == -1 and row['close'] < row['EMA_200']: # close < ema200 is downtrend
            isTrade = True
            stoploss = row['last_high']
            lossdept = abs(row['close'] - stoploss)
            takeprofit = min(row['close'] - 2 * lossdept, row['last_low'])  # Ensure TP is 2x SL
            
            return isTrade, takeprofit, stoploss
    except Exception as e:
        print(f"Error: {e}")
    
    return False, None, None

In [25]:
def transform_data(data):
    df = data.copy()

    # Calculate EMAs
    df['EMA_200'] = df['close'].ewm(span=200, adjust=False).mean()
    df['EMA_50'] = df['close'].ewm(span=50, adjust=False).mean()
    df['EMA_20'] = df['close'].ewm(span=20, adjust=False).mean()

    # Caluculate the difference between the EMAs
    df['diff'] =  df['EMA_20'] - df['EMA_50']

    # Determine the previous difference
    df['diff_prev'] = df['diff'].shift(1)

    # If cross downward, set to -1, if cross upward, set to 1
    df['cross'] = ((df['diff'] > 0) & (df['diff_prev'] < 0)).astype(int) - \
                    ((df['diff'] < 0) & (df['diff_prev'] > 0)).astype(int)

    # Calculate the low and high for the last 50 candles
    df['last_low'] = df['low'].rolling(window=50, min_periods=1).min()
    df['last_high'] = df['high'].rolling(window=50, min_periods=1).max()

    # Determine the trade signals
    df[['isTrade', 'takeprofit', 'stoploss']] = df.apply(is_trade, axis=1, result_type="expand")

    return df
    


In [27]:
## Testing the transform_data function
df =  transform_data(data)
df

,time,volume,open,low,high,close,EMA_200,EMA_50,EMA_20,diff,diff_prev,cross,last_low,last_high,isTrade,takeprofit,stoploss
0,2025-01-03T20:20:00.000000000Z,87,1.03060,1.03048,1.03064,1.03053,1.030530,1.030530,1.030530,0.000000,NaN,0,1.03048,1.03064,False,NaN,NaN
1,2025-01-03T20:21:00.000000000Z,59,1.03051,1.03048,1.03054,1.03053,1.030530,1.030530,1.030530,0.000000,0.000000,0,1.03048,1.03064,False,NaN,NaN
2,2025-01-03T20:22:00.000000000Z,50,1.03055,1.03052,1.03062,1.03062,1.030531,1.030534,1.030539,0.000005,0.000000,0,1.03048,1.03064,False,NaN,NaN
3,2025-01-03T20:23:00.000000000Z,29,1.03061,1.03056,1.03062,1.03057,1.030531,1.030535,1.030542,0.000007,0.000005,0,1.03048,1.03064,False,NaN,NaN
4,2025-01-03T20:24:00.000000000Z,88,1.03056,1.03046,1.03058,1.03047,1.030531,1.030532,1.030535,0.000002,0.000007,0,1.03046,1.03064,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,2025-01-09T08:06:00.000000000Z,246,1.02884,1.02878,1.02920,1.02904,1.030296,1.029581,1.029197,-0.000384,-0.000389,0,1.02836,1.03076,False,NaN,NaN
4995,2025-01-09T08:07:00.000000000Z,201,1.02902,1.02896,1.02924,1.02912,1.030284,1.029563,1.029190,-0.000373,-0.000384,0,1.02836,1.03076,False,NaN,NaN
4996,2025-01-09T08:08:00.000000000Z,163,1.02912,1.02897,1.02918,1.02898,1.030271,1.029540,1.029170,-0.000370,-0.000373,0,1.02836,1.03076,False,NaN,NaN
4997,2025-01-09T08:09:00.000000000Z,218,1.02897,1.02897,1.02930,1.02924,1.030261,1.029528,1.029176,-0.000352,-0.000370,0,1.02836,1.03076,False,NaN,NaN


In [28]:
df[df['cross'] != 0]

,time,volume,open,low,high,close,EMA_200,EMA_50,EMA_20,diff,diff_prev,cross,last_low,last_high,isTrade,takeprofit,stoploss
6,2025-01-03T20:26:00.000000000Z,47,1.03051,1.03041,1.03052,1.03044,1.030530,1.030528,1.030524,-3.866169e-06,1.418844e-06,-1,1.03041,1.03064,True,1.03004,1.03064
37,2025-01-03T20:57:00.000000000Z,102,1.03061,1.03053,1.03067,1.03067,1.030477,1.030408,1.030422,1.472805e-05,-6.352311e-07,1,1.03010,1.03067,True,1.03181,1.03010
103,2025-01-05T22:09:00.000000000Z,3,1.03048,1.03048,1.03049,1.03049,1.030599,1.030691,1.030681,-1.012043e-05,1.792056e-06,-1,1.03048,1.03094,True,1.02959,1.03094
129,2025-01-05T22:36:00.000000000Z,10,1.03072,1.03072,1.03074,1.03074,1.030590,1.030620,1.030622,2.501591e-06,-4.990332e-06,1,1.03037,1.03094,True,1.03148,1.03037
145,2025-01-05T22:52:00.000000000Z,41,1.03048,1.03016,1.03052,1.03020,1.030598,1.030625,1.030608,-1.604046e-05,9.631036e-06,-1,1.03016,1.03092,True,1.02876,1.03092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4862,2025-01-09T05:54:00.000000000Z,48,1.03098,1.03096,1.03104,1.03098,1.031169,1.031024,1.031023,-9.146952e-07,1.800294e-06,-1,1.03083,1.03127,True,1.03040,1.03127
4914,2025-01-09T06:46:00.000000000Z,63,1.03061,1.03060,1.03069,1.03060,1.030830,1.030446,1.030455,8.145685e-06,-8.919318e-07,1,1.02971,1.03070,False,NaN,NaN
4924,2025-01-09T06:56:00.000000000Z,183,1.03026,1.02990,1.03026,1.02991,1.030799,1.030459,1.030449,-9.842356e-06,2.448743e-05,-1,1.02971,1.03084,True,1.02805,1.03084
4943,2025-01-09T07:15:00.000000000Z,116,1.03055,1.03053,1.03084,1.03078,1.030714,1.030394,1.030401,6.494557e-06,-1.769512e-05,1,1.02972,1.03084,True,1.03290,1.02972


#### Function to calculate position price base on account balance 

In [9]:
def calculate_position_size(account_balance, risk_percentage, entry_price, stop_loss_price):
    # Calculate risk amount
    risk_amount = account_balance * risk_percentage /100

     # Calculate position size
    risk_per_unit = abs(entry_price - stop_loss_price)
    if risk_per_unit == 0:
        raise ValueError("Entry price and stop loss price cannot be the same.")

    position_size = risk_amount / risk_per_unit
    return round(position_size)

In [29]:
def test(period, instrument):
    
    raw_data = fetch_data(instrument,period, 5000)
    data = convert_to_df(raw_data)
    df =  transform_data(data)
    in_position, entry = False, False
    account_balance = 1000
    risk = 1 # percentage risk
    stoploss = 0
    takeprofit = 0
    position_size = 0
    transaction = []

    for _, row in df.iterrows():
        # print(row)
        if  in_position == False:
            if row['isTrade']: 
                in_position = True
                entryprice = row['close']
                stoploss = row['stoploss']
                takeprofit = row['takeprofit']
                try:
                    position_size = calculate_position_size(account_balance,risk,entryprice,stoploss)
                except:
                    continue
            
                transaction.append({
                    "type" : "Short" if stoploss > entryprice else "Long",
                    "entry" : entryprice,
                    "SL" : stoploss,
                    "TP" : takeprofit,
                    "size" :position_size,
                    "status" : "incomplete",
                    "winloss" : 0
                })

        else:
            try:
                if transaction[-1]['type'] == "Long":
                    if transaction[-1]['SL'] > row['low'] :
                        transaction[-1]['status'] = "complete"
                        transaction[-1]['winloss'] = transaction[-1]['size'] * (transaction[-1]['SL'] - transaction[-1]['entry'] )
                        account_balance += transaction[-1]['winloss']
                        in_position = False
                    
                    if transaction[-1]['TP'] < row['high']:
                        transaction[-1]['status'] = "complete"
                        transaction[-1]['winloss'] = transaction[-1]['size'] * abs(transaction[-1]['TP'] - transaction[-1]['entry'] )
                        account_balance += transaction[-1]['winloss']
                        in_position = False
                    
                
                if transaction[-1]['type'] == "Short":
                    if transaction[-1]['SL'] < row['high'] :
                        transaction[-1]['status'] = "complete"
                        transaction[-1]['winloss'] = transaction[-1]['size'] * (transaction[-1]['entry'] - transaction[-1]['SL'] )
                        account_balance += transaction[-1]['winloss']
                        in_position = False

                    if transaction[-1]['TP'] > row['low']:
                        transaction[-1]['status'] = "complete"
                        transaction[-1]['winloss'] = transaction[-1]['size'] * abs(transaction[-1]['TP'] - transaction[-1]['entry'] )
                        account_balance += transaction[-1]['winloss']
                        in_position = False
            except:
                transaction = None

   
    df_transaction = pd.DataFrame(transaction)
    df_transaction.to_csv("transaction.csv")
    
    try :
        winrate = len(df_transaction[df_transaction['winloss']>0] ) / len(df_transaction) * 100
    except:
        winrate = 0
    print(f"Pair: {instrument} \tTime :{period} \tBalance: {account_balance:2f} \tWinrate : {winrate:2f}")
    numTrade = len(df_transaction)

    return instrument, period, account_balance, winrate, numTrade


In [30]:
result = []
pairs = ["EUR_USD","USD_JPY","GBP_USD","USD_CAD","AUD_USD","USD_CHF","USD_HKD","EUR_GBP","XAU_USD"]
periods = ["S5", "S10","S15","S30","M1"]


for pair in pairs:
    for period in periods:
        instrument, period, account_balance, winrate, numTrade = test(period,pair)
        result.append({
            "instrument" : instrument,
            "period" : period,
            "balance" : account_balance,
            "numtrade" : numTrade,
            "winrate" : winrate

        })

df = pd.DataFrame(result)
print(df.sort_values("balance",ascending=False))

Pair: EUR_USD 	Time :S5 	Balance: 1017.873750 	Winrate : 36.363636
Pair: EUR_USD 	Time :S10 	Balance: 1007.695350 	Winrate : 34.782609
Pair: EUR_USD 	Time :S15 	Balance: 949.008900 	Winrate : 26.923077
Pair: EUR_USD 	Time :S30 	Balance: 1008.297660 	Winrate : 33.333333
Pair: EUR_USD 	Time :M1 	Balance: 998.213170 	Winrate : 31.578947
Pair: USD_JPY 	Time :S5 	Balance: 1113.511000 	Winrate : 52.631579
Pair: USD_JPY 	Time :S10 	Balance: 1113.261000 	Winrate : 50.000000
Pair: USD_JPY 	Time :S15 	Balance: 998.168000 	Winrate : 33.333333
Pair: USD_JPY 	Time :S30 	Balance: 1017.798000 	Winrate : 36.000000
Pair: USD_JPY 	Time :M1 	Balance: 1039.261000 	Winrate : 40.000000
Pair: GBP_USD 	Time :S5 	Balance: 1146.975340 	Winrate : 52.173913
Pair: GBP_USD 	Time :S10 	Balance: 1028.155580 	Winrate : 38.095238
Pair: GBP_USD 	Time :S15 	Balance: 1038.231540 	Winrate : 39.130435
Pair: GBP_USD 	Time :S30 	Balance: 1070.015130 	Winrate : 45.000000
Pair: GBP_USD 	Time :M1 	Balance: 978.928790 	Winrate : 

In [31]:
df.sort_values('balance', ascending=False)

,instrument,period,balance,numtrade,winrate
37,EUR_GBP,S15,1159.94310,9,88.888889
10,GBP_USD,S5,1146.97534,23,52.173913
19,USD_CAD,M1,1114.90358,8,75.000000
5,USD_JPY,S5,1113.51100,19,52.631579
6,USD_JPY,S10,1113.26100,22,50.000000
35,EUR_GBP,S5,1091.73829,18,50.000000
13,GBP_USD,S30,1070.01513,20,45.000000
18,USD_CAD,S30,1058.68094,27,40.740741
23,AUD_USD,S30,1049.97182,10,50.000000
25,USD_CHF,S5,1049.03142,19,42.105263
